In [1]:
import glob
from dalab import read_pickle
import pandas as pd
import numpy as np
from langdetect import detect
from collections import Counter
from nltk import word_tokenize

import pandas as pd
import numpy as np
from nltk.tokenize import word_tokenize
import spacy
from time import time
import keras
from keras.preprocessing.text import one_hot
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Embedding, Dense, Conv1D, MaxPooling1D, Flatten, Dropout, SimpleRNN, GRU, LSTM
from keras.layers import Input, Dense
from keras.models import Model
from keras.models import Sequential
from matplotlib import pyplot as plt

%matplotlib inline

Using TensorFlow backend.


In [2]:
df = read_pickle('data/20_newsgroup/dataframes/raw_news.pickle')
df = df.sample(frac=1)
df = df.drop_duplicates(subset='text')
df.head()

,label,text
17410,christian,Path: cantaloupe.srv.cs.cmu.edu!rochester!rutg...
4586,baseball,Path: cantaloupe.srv.cs.cmu.edu!das-news.harva...
9607,misc,Newsgroups: talk.politics.misc\nPath: cantalou...
2043,hardware,Path: cantaloupe.srv.cs.cmu.edu!rochester!udel...
8281,med,Xref: cantaloupe.srv.cs.cmu.edu misc.consumers...


In [3]:
# df = pd.read_pickle('data/medium_stories/dataframes/en_lower_stories.pickle').reset_index(drop=True)
# df = df.sample(frac=1)
# df = df.drop_duplicates(subset='story')
# df.head()

In [4]:
for topic in df['label'].unique():
    print(topic, len(df[df['label'] == topic]))

christian 997
baseball 993
misc 2681
hardware 1965
med 994
x 991
electronics 989
guns 964
space 991
motorcycles 999
mideast 973
graphics 987
forsale 992
autos 990
crypt 999
hockey 994
atheism 894


In [5]:
MAXLEN = 1000
VOCAB_SIZE = 20000
TRAIN_SIZE = 10000

In [6]:
nlp = spacy.load('en_core_web_sm')

In [7]:
all_words = word_tokenize(' '.join(df.text.tolist()))
word_counts = Counter(all_words).most_common(VOCAB_SIZE)
words = [w[0] for w in word_counts]

In [8]:
embed_dic = {}
for index, word in enumerate(words):
    if index % 500 == 0: print(index)
    token = nlp(word)
    embed_dic[token.text] = token.vector

0
500
1000
1500
2000
2500
3000
3500
4000
4500
5000
5500
6000
6500
7000
7500
8000
8500
9000
9500
10000
10500
11000
11500
12000
12500
13000
13500
14000
14500
15000
15500
16000
16500
17000
17500
18000
18500
19000
19500


In [9]:
embed_words = pd.DataFrame(embed_dic).T
embed_words.head(10)

,0,1,2,3,4,5,6,7,8,9,...,86,87,88,89,90,91,92,93,94,95
>,3.271235,0.865377,-2.062254,1.412337,-0.318887,1.570737,0.218967,-1.616881,-3.192619,-2.547622,...,0.612776,-0.346440,4.841980,0.257593,-1.508441,5.375728,1.134497,0.064851,-0.028303,2.945163
:,3.070288,0.986614,-1.164513,1.810464,2.261192,-0.716290,0.249038,-1.720148,0.056825,-3.444734,...,-3.068707,0.864790,3.221823,1.004827,-0.291131,6.993183,-0.521152,-2.925200,-0.208749,2.764246
",",0.435715,2.059121,-2.827400,5.559331,-0.635631,-0.084876,1.348762,-1.338799,-2.545363,-4.161990,...,-0.583077,0.877851,4.302979,-2.965968,-0.829183,1.851194,4.129368,-2.011104,-1.491037,0.611928
.,0.017206,1.433102,2.001210,3.140334,1.852441,-0.963332,1.899922,-0.401818,-2.946477,-2.282340,...,-1.228596,0.986874,1.485204,-2.851583,-2.198703,2.439004,3.210545,-2.390744,-1.637757,0.658658
!,-0.438195,-0.151792,0.629396,2.482028,0.040842,0.089026,2.102615,-1.106307,-3.831193,-2.613150,...,-2.212205,0.977764,2.540429,-1.793072,-1.837151,3.268545,3.442772,-2.944798,-1.267171,0.232743
the,-1.553254,-2.913822,-4.319618,2.272106,-1.109262,1.648155,-2.616732,-2.518888,-2.483821,-2.783311,...,3.250357,-1.727508,4.852344,-0.260655,3.556051,6.408743,-0.459191,1.640075,-0.309467,-1.231600
--,-0.836566,1.029746,-1.545806,0.859156,3.777660,-0.975338,0.222508,-0.303487,-1.498767,-2.576878,...,-2.571487,1.666716,3.351637,1.144979,-1.691302,5.329454,0.757731,-3.216542,-1.397676,2.549656
@,1.050108,0.208937,0.347154,2.047950,1.639953,1.493342,1.208333,0.460561,-2.761452,-2.527010,...,-0.653786,0.561741,2.965085,1.466421,0.470562,3.923660,4.240563,0.821820,-1.435084,3.934670
to,-0.061328,1.449228,-3.866241,0.745157,-1.463142,0.375383,-2.931649,-0.653668,-2.569345,-3.578854,...,-0.515135,2.122284,1.689541,3.462939,1.066636,7.533019,2.215232,-2.120562,-2.444374,2.538816
),0.479581,0.744241,-0.826027,0.408746,1.193886,-1.085895,-0.922773,-0.325466,-1.257374,-5.033413,...,1.058636,2.931694,3.694527,-1.901443,-2.631643,4.643446,3.615112,-3.458215,0.200469,3.789077


In [10]:
padding = pd.DataFrame({'<PAD>': np.zeros(shape=[1,embed_words.shape[1]])[0]}).T
embed_matrix = padding.append(embed_words)
embed_matrix.head()

,0,1,2,3,4,5,6,7,8,9,...,86,87,88,89,90,91,92,93,94,95
<PAD>,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
>,3.271235,0.865377,-2.062254,1.412337,-0.318887,1.570737,0.218967,-1.616881,-3.192619,-2.547622,...,0.612776,-0.346440,4.841980,0.257593,-1.508441,5.375728,1.134497,0.064851,-0.028303,2.945163
:,3.070288,0.986614,-1.164513,1.810464,2.261192,-0.716290,0.249038,-1.720148,0.056825,-3.444734,...,-3.068707,0.864790,3.221823,1.004827,-0.291131,6.993183,-0.521152,-2.925200,-0.208749,2.764246
",",0.435715,2.059121,-2.827400,5.559331,-0.635631,-0.084876,1.348762,-1.338799,-2.545363,-4.161990,...,-0.583077,0.877851,4.302979,-2.965968,-0.829183,1.851194,4.129368,-2.011104,-1.491037,0.611928
.,0.017206,1.433102,2.001210,3.140334,1.852441,-0.963332,1.899922,-0.401818,-2.946477,-2.282340,...,-1.228596,0.986874,1.485204,-2.851583,-2.198703,2.439004,3.210545,-2.390744,-1.637757,0.658658


In [11]:
word_index = {j:i+1 for i,j in enumerate(embed_matrix.index.tolist()[1:])}
tokenizer = Tokenizer()
tokenizer.word_index = word_index
sequences = tokenizer.texts_to_sequences(df.text)
data = pad_sequences(sequences, maxlen=MAXLEN)

In [12]:
random_matrix = np.random.randn(embed_matrix.shape[0], embed_matrix.shape[1])
random_matrix[0] = np.zeros([1, embed_matrix.shape[1]])

In [13]:
onehot = pd.get_dummies(df['label'])
target_labels = onehot.columns
target = onehot.as_matrix()
target

/Users/rodrigonader/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  This is separate from the ipykernel package so we can avoid doing imports until


array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 1, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [1, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 1, 0, 0],
       [0, 0, 0, ..., 0, 1, 0]], dtype=uint8)

In [14]:
x_train = data[:TRAIN_SIZE]
x_test = data[TRAIN_SIZE:]

y_train = target[:TRAIN_SIZE]
y_test = target[TRAIN_SIZE:]

In [15]:
embedding_layer = Embedding(len(embed_matrix), len(embed_matrix.columns), input_length=MAXLEN, weights=[embed_matrix],
                           trainable=False)

sequence_input = Input(shape=(MAXLEN,), dtype='int32')
embedded_sequences = embedding_layer(sequence_input)

x = Conv1D(128, 5, activation='relu')(embedded_sequences)
x = MaxPooling1D(5)(x)
x = Conv1D(128, 5, activation='relu')(x)
x = MaxPooling1D(5)(x)
x = Dropout(0.2)(x)
x = Conv1D(128, 5, activation='relu')(x)
x = MaxPooling1D(35)(x)  # global max pooling
x = Flatten()(x)
x = Dense(128, activation='relu')(x)
output = Dense(target.shape[1], activation='softmax')(x)

model = Model(sequence_input, output)

opt = keras.optimizers.Adam(lr=1e-3, decay=1e-5)

model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['acc'])
model.fit(data, target, validation_split=0.2, epochs=2, batch_size=128)

W0828 16:54:48.156697 4570703296 deprecation_wrapper.py:119] From /Users/rodrigonader/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0828 16:54:48.197672 4570703296 deprecation_wrapper.py:119] From /Users/rodrigonader/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0828 16:54:48.210897 4570703296 deprecation_wrapper.py:119] From /Users/rodrigonader/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0828 16:54:48.228646 4570703296 deprecation_wrapper.py:119] From /Users/rodrigonader/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:174: The name tf.get_default_session is deprecated. Please use tf.compat.v1.get_de

Train on 15514 samples, validate on 3879 samples
Epoch 1/2
15514/15514 [==============================] - 66s 4ms/step - loss: 2.3759 - acc: 0.2643 - val_loss: 0.9013 - val_acc: 0.6677
Epoch 2/2
15514/15514 [==============================] - 65s 4ms/step - loss: 0.4241 - acc: 0.8660 - val_loss: 0.1236 - val_acc: 0.9652


array([[  0,   0,   0, ...,   0,   0, 108],
       [  0,   0,   0, ..., 150, 109, 101],
       [  0,   0,   0, ..., 108, 109, 101],
       ...,
       [  0,   0,   0, ...,  36,  75, 205],
       [  0,   0,   0, ..., 107,  42,  20],
       [  0,   0,   0, ...,   0,   1, 127]], dtype=int32)

In [16]:
# model.summary()